# **Diplomová práce - výpočty**
### **Jméno:** Alois Primes
### **Název práce:** MODULARNÍ HORIZONTÁLNÍ KOTEL – HRSG
### **Zadání**
**Stručná charakteristika problematiky úkolu:**

Práce je zaměřena na návrh horizontálního kotle využívajícího spaliny za plynovou spalovací turbínou pro produkci páry k dalšímu energetickému využití. Kotel by měl být moderní konstrukce, zahrnující
maximální prefabrikaci transportních bloků (modulů), která je ovlivněna podmínkami dopravního profilu na místo stavby. Kotel bude řešen jako modulární, s horizontálním prouděním spalin, konvekčním
výparníkem, výhřevnými plochami ohříváku vody (ekonomizéry) a přehříváky páry s regulací teploty páry vstřikem napájecí vody, odebírané mezi dvěma ekonomizéry.

**Základní parametry kotle jsou:** parní výkon 76 t/h, tlak 3,1 MPa, výstupní teplota 370 °C. Teplota
napájecí vody kotle je 71°C. Další parametry kotle a odpadních spalin budou upřesněny v rámci
realizace práce.

**Cíle diplomové práce:**

Zpracovat tepelný výpočet kotle.
Zpracovat rozměrový návrh kotle, včetně základní výkresové dokumentace kotle a dispozičního
řešení.
Vyčíslit dosaženou účinnost kotle při vztažné teplotě 25 °C, použijte přímou metodu.
Zpracovat diagram průběhu teploty pracovního média a spalin a stanovit bilanci vstřiku.

In [4]:
# Instalace a aktualizace modulů

# !pip install --upgrade pip
# !pip install --upgrade forallpeople
# !pip install --upgrade handcalcs
# !pip install --upgrade pytexit
# !pip install --upgrade pyXSteam
# !pip install --upgrade CoolProp

In [5]:
# Import knihoven

from CoolProp.CoolProp import PropsSI
import handcalcs.render
from math import pi, e
from pyXSteam.XSteam import XSteam

In [6]:
# Nastavení prostředí a modulů

%decimal_separator ,
steam = XSteam(XSteam.UNIT_SYSTEM_MKS) # Jednotky: m/kg/sec/°C/bar/W

## Definice pomocných funkcí

In [7]:
def density(prvek, teplota, tlak):

	"""
	Zadej požadavek ve formátu density('prvek', teplota [K], tlak [BAR])
	Příklad rho_CO2 = density('CO2', 293.15, 5.5)
	Vypočte hustotu CO2 pro teplotu 20 °C (293.15 K) a 5.5 baru (550 000 Pa)
    JEDNOTKA: kg/m^3
	"""

	rho = PropsSI('D', 'T', teplota, 'P', tlak*100000, prvek)
	return rho

In [8]:
def conductivity(prvek, teplota, tlak):

	"""
	Zadej požadavek ve formátu conductivity('prvek', teplota [K], tlak [bar])
	Příklad lambda_CO2 = conductivity('CO2', 293.15, 1.01325)
	Vypočte vodivost CO2 pro teplotu 20 °C (293.15 K) a 1.01325 baru (101 325 Pa)
    JEDNOTKA: W/m*K
	"""

	lambda_prvek = PropsSI('conductivity', 'T', teplota, 'P', tlak*100000, prvek)
	return lambda_prvek

In [9]:
def enthalpy(prvek, teplota, tlak):

	"""
	Zadej požadavek ve formátu enthalpy('prvek', teplota [K], tlak [bar])
	Příklad h_CO2 = enthalpy('CO2', 293.15, 1.01325)
	Vypočte entalpii CO2 pro teplotu 20 °C (293.15 K) a 1.01325 baru (101 325 Pa)
	JEDNOTKA: J/kg
	"""

	h_prvek = PropsSI('H', 'T', teplota, 'P', tlak*100000, prvek)
	return h_prvek

In [10]:
def viscosity(prvek, teplota, tlak):

	"""
	Zadej požadavek ve formátu viscosity('prvek', teplota [K], tlak [bar])
	Příklad eta_CO2 = viscosity('CO2', 293.15, 1.01325)
	Vypočte viskozitu CO2 pro teplotu 20 °C (293.15 K) a 1.01325 baru (101 325 Pa)
	JEDNOTKA: Pa.s
	"""

	eta_prvek = PropsSI('viscosity', 'T', teplota, 'P', tlak*100000, prvek)
	return eta_prvek

In [11]:
def cp(prvek, teplota, tlak):

	"""
	Zadej požadavek ve formátu cp('prvek', teplota [K], tlak [bar])
	Příklad cp_CO2 = cp('CO2', 293.15, 1.01325)
	Vypočte měrnou tepelnou kapacitu CO2 pro teplotu 20 °C (293.15 K) a 1.01325 baru (101 325 Pa)
	JEDNOTKA: J/kg*K
	"""

	cp_prvek = PropsSI('C', 'T', teplota, 'P', tlak*100000, prvek)
	return cp_prvek

In [12]:
def kin_viscosity(prvek, teplota, tlak):

	"""
	Zadej požadavek ve formátu kin_viscosity('prvek', teplota [K], tlak [bar])
	Příklad nu_CO2 = kin_viscosity('CO2', 293.15, 1.01325)
	Vypočte kinematickou viskozitu CO2 pro teplotu 20 °C (293.15 K) a 1.01325 baru (101 325 Pa)
	JEDNOTKA: m^2/s
	"""
	eta_prvek = PropsSI('viscosity', 'T', teplota, 'P', tlak*100000, prvek)
	rho_prvek = PropsSI('D', 'T', teplota, 'P', tlak*100000, prvek)
	nu_prvek = eta_prvek/rho_prvek
	return nu_prvek

## Základní parametry kotle

In [13]:
%%render params 2
m_p = 76        #t/h
p_out = 3.1     #MPa
t_out = 370     #°C
t_nv = 71       #°C
m_vst = 0.36    #kg/s
t_vst = 217     #°C

<IPython.core.display.Latex object>

## Parametry spalin ze spalovací turbíny

In [14]:
%%render params 3
m_sp = 130.1    #kg/s
t_sp = 562.0    #°C

# Objemové složení spalin
x_N2 = 0.7457
x_O2 = 0.1289
x_H2O = 0.0801
x_CO2 = 0.0364
x_Ar = 0.0089

<IPython.core.display.Latex object>

In [15]:
%%render params 3

# Molární hmotnost složek spalin (z podkladů)
M_N2 = 28.0134     #kg/mol
M_O2 = 31.9988     #kg/mol
M_H2O = 18.0152    #kg/mol
M_CO2 = 44.0095    #kg/mol
M_Ar = 39.9480     #kg/mol

<IPython.core.display.Latex object>

In [16]:
%%render params

# Hustota složek spalin (z podkladů)
rho_N2 = 1.2504     #kg/m3
rho_O2 = 1.4290     #kg/m3
rho_H2O = 0.8038    #kg/m3
rho_CO2 = 1.9772    #kg/m3
rho_Ar = 1.7840     #kg/m3

<IPython.core.display.Latex object>

In [17]:
%%render params 3

# Měrný objem složek spalin (z podkladů)
v_N2 = 22.403     #m3/kg
v_O2 = 22.392     #m3/kg
v_H2O = 22.414    #m3/kg
v_CO2 = 22.258    #m3/kg
v_Ar = 22.393     #m3/kg

<IPython.core.display.Latex object>

#### Bod 1 (Konec druhého přehříváku, výstupní pára)

In [18]:
%%render params
t_1 = t_out                     #°C
p_1 = p_out                     #MPa
i_1 = steam.h_pt(p_1*10,t_1)    #kJ/kg
v_1 = steam.v_pt(p_1*10,t_1)     #m3/kg

<IPython.core.display.Latex object>

#### Bod 2 (Začátek druhého přehříváku, po vstřiku) DOPOČÍTAT!!!

In [19]:
%%render params
t_2 = t_out                     #°C
p_2 = p_out                     #MPa
i_2 = steam.h_pt(p_2*10,t_2)    #kJ/kg
v_2 = steam.v_pt(p_2*10,t_2)    #m3/kg

<IPython.core.display.Latex object>

#### Bod 3 (Konec prvního přehříváku, před vstřikem)

In [20]:
%%render params
t_3 = 348                       #°C
p_3 = 3.33                      #MPa
i_3 = steam.h_pt(p_3*10,t_3)    #kJ/kg
v_3 = steam.v_pt(p_3*10,t_3)    #m3/kg

<IPython.core.display.Latex object>

#### Bod 4 (Začátek prvního přehříváku, po druhém výparníku)

In [21]:
%%render params
p_4 = 3.38                      #MPa
t_4 = steam.tsat_p(p_4*10)      #°C
i_4 = steam.hV_p(p_4*10)        #kJ/kg
v_4 = steam.vV_p(p_4*10)        #m3/kg

<IPython.core.display.Latex object>

#### Bod 5 (začátek prvního výparníku, kde je sytá kapalina)

In [22]:
%%render params
p_5 = 3.38                      #MPa
t_5 = steam.tsat_p(p_5*10)      #°C
i_5 = steam.hL_p(p_5*10)        #kJ/kg
v_5 = steam.vL_p(p_5*10)        #m3/kg

<IPython.core.display.Latex object>

#### Bod 6 (????)